<a href="https://colab.research.google.com/github/sytrinh/machine-learning-from-scratch/blob/main/machine_learning_algorithms/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Regression Problem

Let's assume that we want to predict $y$ from a list of independent variables $x_1, x_2, ..., x_n$, and their relationship can be expressed by a function $f_t$ as

$$y = f_t(\mathbf{x}),$$

where $\mathbf{x} = [x_1, x_2, ..., x_N]$ is a row vector. The true function $f_t$ is generally unknown, but we consider a function $f$ that is approximately $f_t$ that can generate predictions close to the grounth truth ($y$), given by 

$$y \approx \hat{y} = f(\mathbf{x}) = \mathbf{x} \mathbf{w},$$

where $\mathbf{w} = [w_1, w_2, ..., w_N]^T$ is a column vector of model parameters. 

If we want to account for the bias, $\mathbf{x}$ and $\mathbf{w}$ are given as follows:

$$\mathbf{x} = [1, x_1, x_2, ..., x_N]$$
$$\mathbf{w} = [w_0, w_1, w_2, ..., w_N]^T$$

### Data

Now assume that each predictor $x_i$ has $M$ samples, then all the values of $N$ features can be put into a $M \times N$ matrix $\mathbf{X}$.

$$\mathbf{X} = [\mathbf{1}, \mathbf{x}_1, \mathbf{x}_2, ..., \mathbf{x}_N],$$
where $\mathbf{x}_i$ is an $M$-dimensional column vector containing $M$ samples of predictor $x_i$.

$\mathbf{y} = [y_1, y_2, ... , y_M]^T$ is a columns vector of true values of the outcome (from data), and $\hat{\mathbf{y}}$ is a vector of predicted values.

We have:

$$\mathbf{y} \approx \hat{\mathbf{y}} = \mathbf{X} \mathbf{w}$$

### Loss function

$$\mathcal{L}(\mathbf{w}) = \frac{1}{2M}\sum_{j=1}^M (y_j - \mathbf{x}_j \mathbf{w})^2 = \frac{1}{2M} || \mathbf{y} - \mathbf{X} \mathbf{w}||_2^2,$$

where $\mathbf{x}_j$ is a sample vector (a row of $\mathbf{X}$) and $(y_j - \mathbf{x}_j \mathbf{w})$ is the error between the true and predicted values. The loss function is the average squared errors over $M$ samples.

To find $f$, we have to find $\mathbf{w}$ so that the loss function, $\mathcal{L}(\mathbf{w})$, has the lowest value. In other words, to find $\mathbf{w}$ that minimize the average error between the true and predicted values. We need to find $\mathbf{w}^*$ as:

$$\mathbf{w}^* = \arg\min_{\mathbf{w}} \mathcal{L}(\mathbf{w})$$

### Solution of the linear regression problem

We can find $\mathbf{w}$ by solving an optimization problem. Since the loss function is a quadratic function, we find $\mathbf{w}$ by solving:
$$\frac{\partial{\mathcal{L}(\mathbf{w})}}{\partial{\mathbf{w}}} = 0$$

Solution:
$$\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$$

If $(\mathbf{X}^T \mathbf{X})$ is not invertiable, then we can use the pseudo inverse concept. In this case:
$$\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{†} \mathbf{X}^T \mathbf{y},$$

where $(\mathbf{X}^T \mathbf{X})^{†}$ is the pseudo inverse of matrix $(\mathbf{X}^T \mathbf{X})$

### Numerical optimization

Another way to solve the optimization problem is using a numerical method such as gradient descent. We will update the weights using gradient descent algorihtm:

$$\mathbf{w} := \mathbf{w} - \alpha \frac{\partial{\mathcal{L}(\mathbf{w})}}{\partial{\mathbf{w}}} = \mathbf{w} - \alpha \nabla_{\mathbf{w}}\mathcal{L}(\mathbf{w})$$

We have:

$$\nabla_{\mathbf{w}}\mathcal{L}(\mathbf{w}) = \frac{1}{M} \mathbf{X}^T(\mathbf{X} \mathbf{w}-\mathbf{y}) = \frac{1}{M} \mathbf{X}^T(\hat{\mathbf{y}}-\mathbf{y})$$

## Implementation

In [28]:
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.datasets import make_regression 

# creating the data set
X, y = make_regression(n_samples=100, n_features=2, n_targets=1, noise=20, random_state=1)

# splitting training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1) 

In [29]:
class MyLR1:
  """
  Linear Regression implementation with exact formula
  """

  def __init__(self):
    self.params = {
        'W': None
    }

  def train(self, X_train, y_train):
    y_train = y_train.copy().reshape((-1,1))
    assert X_train.shape[0] == y_train.shape[0]

    one = np.ones((X_train.shape[0], 1))
    # X = np.concatenate((one, X_train), axis = 1)
    X = X_train.copy()
    X = np.hstack((one, X)) # to include bias

    # W =  np.dot(np.linalg.pinv(np.dot(X.T, X)), np.dot(X.T, y_train))
    W = np.linalg.pinv(X.T @ X) @ X.T @ y_train
    assert W.shape == (X.shape[1],1)
    self.params['W'] = W

  def predict(self, X_test):
    one = np.ones((X_test.shape[0], 1))
    X = np.hstack((one, X_test))
    y_preds = X @ self.params['W']
    return y_preds


mylr1 = MyLR1()
mylr1.train(X_train, y_train)
mylr1.predict(X_test)
mylr1.params

{'W': array([[ 3.03731077],
        [31.64575094],
        [80.925791  ]])}

In [30]:
class MyLR2:

  """
  Linear Regression implementation using gradient descent
  """

  def __init__(self, learning_rate=0.01, n_iter=1000):
    self.learning_rate = learning_rate
    self.n_iter = n_iter
    self.params = {
        'W': None,
    }

  def init_params(self, n_weights):
    self.params['W'] = np.zeros((n_weights, 1))


  def gradient_descent(self, M, W, X, y):
    W -= 1/M * self.learning_rate * (X.T @ (X @ W - y))
    return W

  def train(self, X_train, y_train):
    y = y_train.copy().reshape((-1,1))
    assert X_train.shape[0] == y_train.shape[0]

    one = np.ones((X_train.shape[0], 1))
    X = X_train.copy()
    X = np.hstack((one, X)) # to include bias

    M, N = X.shape
    self.init_params(N)

    W = self.params['W']
    for i in range(self.n_iter):
      W = self.gradient_descent(M, W, X, y)
    self.params['W'] = W

  def predict(self, X_test):
    one = np.ones((X_test.shape[0], 1))
    X = np.hstack((one, X_test))
    y_preds = X @ self.params['W']
    return y_preds

mylr2 = MyLR2()
mylr2.train(X_train, y_train)
mylr2.params

{'W': array([[ 3.09489998],
        [31.61578819],
        [80.83326253]])}

In [31]:
class MyLR3:

  """
  Linear Regression implementation using gradient descent but sperating weights and bias
  """

  def __init__(self, learning_rate=0.01, n_iter=1000):
    self.learning_rate = learning_rate
    self.n_iter = n_iter
    self.params = {
        'W': None,
        'b': None
    }

  def init_params(self, n_weights):
    self.params['W'] = np.zeros((n_weights, 1))
    self.params['b'] = 0

  def gradient_descent(self, X, y):
    M, N = X.shape
    W, b = self.params['W'], self.params['b']
    for i in range(self.n_iter):
      y_pred = X @ W + b
      W -= 1/M * self.learning_rate * (X.T @ (y_pred - y))
      b -= 1/M * self.learning_rate * np.sum(y_pred-y) # In fact it is them same as b.T @ (y_pred-y) when writing b as a vector
    self.params['W'] = W
    self.params['b'] = b

  def train(self, X_train, y_train):
    y = y_train.copy().reshape((-1,1))
    X = X_train.copy()
    assert X.shape[0] == y.shape[0]

    self.init_params(X.shape[1])
    self.gradient_descent(X, y)

  def predict(self, X_test):
      y_preds = np.dot(X_test, self.params["W"]) + self.params["b"]
      return y_preds

mylr3 = MyLR3()
mylr3.train(X_train, y_train)
mylr3.params

{'W': array([[31.61578819],
        [80.83326253]]), 'b': 3.094899980947451}

In [27]:
# Using sklearn

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(lin_reg.coef_)
print(lin_reg.intercept_)

[31.64575094 80.925791  ]
3.037310773722737


In [52]:
# Compare results

mse1 = np.mean((mylr1.predict(X_test).flatten() - y_test)**2)
mse2 = np.mean((mylr2.predict(X_test).flatten() - y_test)**2)
mse3 = np.mean((mylr3.predict(X_test).flatten() - y_test)**2)
mse4 = np.mean((lin_reg.predict(X_test) - y_test)**2)
print(mse1, mse2, mse3, mse4)

641.7771707649288 642.1429652605785 642.1429652605786 641.7771707649296
